<a href="https://colab.research.google.com/github/alzatem204/Lab_Biosenales/blob/main/P3_Parte_2_Python_G7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Práctica 3 - Parte 2 - Comparación estadística de Energía de señales EEG
Análisis estadístico de señales - Grupo 7

Valentina Garcia Obando C.C.1000539432


In [25]:
from google.colab import drive
# Montar Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import os
from scipy.io import loadmat
import numpy as np
import pandas as pd

In [31]:
# Listar archivos en la carpeta "Bioseñales" dentro de "My Drive"
ruta_carpeta = '/content/drive/My Drive/Bioseñales/'
archivos = os.listdir(ruta_carpeta)
print(archivos)

['datos_senales_datos_parkinson_cursos.zip']


In [8]:
# Ruta al archivo comprimido en tu Google Drive
ruta_zip = '/content/drive/My Drive/Bioseñales/datos_senales_datos_parkinson_cursos.zip'

# Descomprimir el archivo en una carpeta específica en Colab
!unzip -o "{ruta_zip}" -d "/content/datos"

Archive:  /content/drive/My Drive/Bioseñales/datos_senales_datos_parkinson_cursos.zip
  inflating: /content/datos/control/C001R_EP_reposo.mat  
  inflating: /content/datos/control/C002_EP_reposo.mat  
  inflating: /content/datos/control/C004_EP_reposo.mat  
  inflating: /content/datos/control/C005_EP_reposo_Repetido.mat  
  inflating: /content/datos/control/C006_EP_reposo.mat  
  inflating: /content/datos/control/C007_EP_reposo.mat  
  inflating: /content/datos/control/C010_EP_reposo.mat  
  inflating: /content/datos/control/C011_EP_reposo.mat  
  inflating: /content/datos/control/C012_EP_reposo.mat  
  inflating: /content/datos/control/C013_EP_reposo.mat  
  inflating: /content/datos/control/C015_EP_reposo.mat  
  inflating: /content/datos/control/C018_EP_reposo.mat  
  inflating: /content/datos/control/C019_EP_reposo.mat  
  inflating: /content/datos/control/C020_EP_reposo.mat  
  inflating: /content/datos/control/C021_EP_reposo.mat  
  inflating: /content/datos/control/C023_EP_repos

In [32]:
# Listar los archivos descomprimidos en la carpeta especificada
archivos_descomprimidos = os.listdir('/content/datos')
print(archivos_descomprimidos)

['control', 'parkinson']


In [10]:
# Listar archivos en la carpeta 'control'
ruta_control = '/content/datos/control/'
archivos_control = os.listdir(ruta_control)
print("Archivos en la carpeta 'control':")
print(archivos_control)

# Listar archivos en la carpeta 'parkinson'
ruta_parkinson = '/content/datos/parkinson/'
archivos_parkinson = os.listdir(ruta_parkinson)
print("Archivos en la carpeta 'parkinson':")
print(archivos_parkinson)

Archivos en la carpeta 'control':
['C036_EP_reposo.mat', 'C018_EP_reposo.mat', 'C031_EP_reposo.mat', 'C001R_EP_reposo.mat', 'C025_EP_reposo.mat', 'C012_EP_reposo.mat', 'C045_EP_reposo.mat', 'C044_EP_reposo.mat', 'C021_EP_reposo.mat', 'C033R_EP_reposo.mat', 'C006_EP_reposo.mat', 'C030_EP_reposo.mat', 'C015_EP_reposo.mat', 'C011_EP_reposo.mat', 'C049_EP_reposo.mat', 'C010_EP_reposo.mat', 'C037_EP_reposo.mat', 'C040_EP_reposo.mat', 'C004_EP_reposo.mat', 'C002_EP_reposo.mat', 'C034_EP_reposo.mat', 'C019_EP_reposo.mat', 'C013_EP_reposo.mat', 'C027_EP_reposo.mat', 'C020_EP_reposo.mat', 'C026_EP_reposo.mat', 'C029_EP_reposo.mat', 'C028_EP_reposo.mat', 'C007_EP_reposo.mat', 'C042_EP_reposo.mat', 'C023_EP_reposo.mat', 'C024_EP_reposo.mat', 'C005_EP_reposo_Repetido.mat', 'C046_EP_reposo.mat', 'C041_EP_reposo.mat', 'C032_EP_reposo.mat']
Archivos en la carpeta 'parkinson':
['P046_EP_reposo.mat', 'P005_EP_reposo.mat', 'P017_EP_reposo.mat', 'P034_EP_reposo.mat', 'P041_EP_reposo.mat', 'P007_EP_reposo

In [11]:
import scipy.io

# Cargar un archivo de ejemplo del grupo control
ruta_archivo_control = '/content/datos/control/C001R_EP_reposo.mat'
datos_control = scipy.io.loadmat(ruta_archivo_control)

# Explorar el contenido del archivo para encontrar la clave con las señales
print(datos_control.keys())

dict_keys(['__header__', '__version__', '__globals__', 'data'])


In [27]:
from scipy.io import loadmat

# Función para cargar todos los archivos .mat en una carpeta
def cargar_datos_carpeta(ruta_carpeta):
    datos = [] # Lista para almacenar los datos cargados
    # Iterar sobre cada archivo en la carpeta
    for archivo in os.listdir(ruta_carpeta):
        # Comprobar si el archivo tiene la extensión .mat
        if archivo.endswith('.mat'):
            try:
                # Crear, cargar y añadir a la lista de datos
                ruta_archivo = os.path.join(ruta_carpeta, archivo)
                contenido = loadmat(ruta_archivo)
                if 'data' in contenido:
                    datos.append(contenido['data'])
                else:
                    print(f"Advertencia: 'data' no encontrado en {archivo}")
            except Exception as e:
                print(f"Error al cargar {archivo}: {e}")
    return datos

# Cargar datos de ambas carpetas
datos_control = cargar_datos_carpeta(ruta_control)
datos_parkinson = cargar_datos_carpeta(ruta_parkinson)

In [28]:
# Definir una función para calcular la energía promedio por canal
def calcular_energia_promedio_por_canal(datos):
    """
    Calcula la energía promedio de cada canal para múltiples épocas.

    Parámetros:
    datos (list of np.array): Lista de matrices de datos, donde cada matriz representa una época
                              y cada fila una señal de un canal.

    Retorna:
    np.array: Un array con la energía promedio de cada canal.
    """
    energias = [] # Lista para almacenar la energía promedio por canal

    for dato in datos:
        # Asegurarse de que 'dato' es una matriz con múltiples épocas
        if len(dato.shape) == 3:  # Si la forma es (n_épocas, n_canales, n_muestras)
            # Calcular la energía para cada época
            energias_epocas = np.sum(np.square(dato), axis=2)  # Energía por canal y época
            # Calcular la energía promedio por canal
            energia_promedio_canal = np.mean(energias_epocas, axis=0)  # Energía promedio por canal
        else:
            raise ValueError("El formato de los datos no es el esperado. Se esperaba una matriz 3D.")

        energias.append(energia_promedio_canal)

    return np.array(energias)

# Ejemplo de uso:
# Suponiendo que cada 'dato' en 'datos_control' y 'datos_parkinson' es una matriz de forma (n_épocas, n_canales, n_muestras)
energias_promedio_control = calcular_energia_promedio_por_canal(datos_control)
energias_promedio_parkinson = calcular_energia_promedio_por_canal(datos_parkinson)

print("Energía promedio por canal para el grupo control:")
print(energias_promedio_control)

print("Energía promedio por canal para el grupo Parkinson:")
print(energias_promedio_parkinson)

Energía promedio por canal para el grupo control:
[[  931.92612562   885.3732343    845.10991271 ...   814.39582653
    846.60060952   885.64221222]
 [10035.06608632  9923.42471733  9807.64939457 ...  9902.13573434
  10192.48422868 10461.0861976 ]
 [ 2146.41157097  2094.93772857  2049.44333565 ...  1944.82743429
   1993.4643876   2043.061663  ]
 ...
 [ 2717.00565593  2648.98358239  2586.61313419 ...  2713.05325593
   2771.94832824  2832.81765796]
 [ 4810.64715238  4676.180767    4552.60004518 ...  4472.07729174
   4665.66149398  4873.60185822]
 [ 3991.74329195  3892.84742293  3793.73481074 ...  3371.47620814
   3467.76800586  3570.18597871]]
Energía promedio por canal para el grupo Parkinson:
[[ 4103.06454151  4013.33885474  3927.10973936 ...  3639.3603839
   3692.82600684  3754.1042093 ]
 [11663.71697423 11510.1398693  11359.89981034 ... 10447.71609752
  10628.92349123 10803.31806116]
 [ 1681.44950401  1601.00389089  1529.81277437 ...  1625.78145187
   1693.32489134  1771.66403707]
 .

In [29]:
import numpy as np
import pandas as pd

# Definir una función para crear un DataFrame a partir de la energía promedio por canal
def crear_dataframe(energia, archivos):
    """
    Crea un DataFrame a partir de la energía promedio por canal.

    Parámetros:
    energia (np.array): Un array con la energía promedio de cada canal para cada sujeto.
    archivos (list): Lista de nombres de archivos para usar como índices en el DataFrame.

    Retorna:
    pd.DataFrame: DataFrame con energía promedio por canal.
    """
    num_canales = energia.shape[1]
    # Obtener el número de canales
    # Crear un diccionario para los datos del DataFrame
    datos_df = {
        f'canal_{i+1}': energia[:, i] for i in range(num_canales)
    }
    df = pd.DataFrame(datos_df, index=[archivo.split('_')[0] for archivo in archivos])
    return df

# Crear DataFrames para el grupo control
df_control = crear_dataframe(energias_promedio_control, os.listdir(ruta_control))
# Crear DataFrames para el grupo Parkinson
df_parkinson = crear_dataframe(energias_promedio_parkinson, os.listdir(ruta_parkinson))

# Mostrar las primeras filas de los DataFrames
print("DataFrame para el grupo control:")
print(df_control.head())

print("DataFrame para el grupo Parkinson:")
print(df_parkinson.head())

DataFrame para el grupo control:
            canal_1       canal_2       canal_3       canal_4       canal_5  \
C036     931.926126    885.373234    845.109913    811.748349    785.634075   
C018   10035.066086   9923.424717   9807.649395   9691.877363   9579.937131   
C031    2146.411571   2094.937729   2049.443336   2011.693885   1983.086644   
C001R   2529.730308   2385.720469   2251.403593   2130.312696   2025.449608   
C025   12186.303230  11838.696857  11478.116786  11114.600749  10758.482524   

            canal_6       canal_7      canal_8      canal_9     canal_10  ...  \
C036     766.854337    755.255619   750.471335   751.958795   759.038563  ...   
C018    9475.098071   9379.864313  9295.833098  9223.629361  9162.922789  ...   
C031    1964.566228   1956.569963  1959.005209  1971.258539  1992.239107  ...   
C001R   1939.160414   1873.037403  1827.856441  1803.558561  1799.267433  ...   
C025   10419.906981  10108.334137  9832.072410  9597.875614  9410.621125  ...   

     

In [30]:
import pandas as pd
from scipy import stats

def verificar_normalidad(datos):
    """
    Verifica la normalidad de los datos usando la prueba de Shapiro-Wilk.
    """

    # Realizar la prueba de Shapiro-Wilk y obtener el valor p
    _, p_value = stats.shapiro(datos)
    # Retornar True si el valor p es mayor que 0.05 (indica normalidad)
    return p_value > 0.05

def verificar_homocedasticidad(grupo1, grupo2):
    """
    Verifica la homocedasticidad usando la prueba de Levene.
    """
    # Realizar la prueba de Levene y obtener el valor p
    _, p_value = stats.levene(grupo1, grupo2)
    # Retornar True si el valor p es mayor que 0.05 (indica homocedasticidad)
    return p_value > 0.05

def realizar_prueba_t(grupo1, grupo2, var_equal=True):
    """
    Realiza la prueba t para muestras independientes.
    """
    # Realizar la prueba t y obtener el valor p
    _, p_value = stats.ttest_ind(grupo1, grupo2, equal_var=var_equal)
    return p_value

def realizar_prueba_mann_whitney(grupo1, grupo2):
    """
    Realiza la prueba U de Mann-Whitney.
    """
    # Realizar la prueba U de Mann-Whitney y obtener el valor p
    _, p_value = stats.mannwhitneyu(grupo1, grupo2, alternative='two-sided')
    return p_value

def analizar_canales(df_control, df_parkinson):
    """
    Analiza si hay diferencias significativas entre canales de los grupos control y Parkinson.
    """
    resultados = []# Lista para almacenar los resultados
    num_canales = df_control.shape[1]  # Número de canales en el DataFrame

    for canal in range(num_canales):
        # Extraer los datos de control y Parkinson para el canal actual
        datos_control = df_control.iloc[:, canal]
        datos_parkinson = df_parkinson.iloc[:, canal]

        # Verificar normalidad
        normalidad_control = verificar_normalidad(datos_control)
        normalidad_parkinson = verificar_normalidad(datos_parkinson)

        # Verificar homocedasticidad si ambos grupos son normales
        if normalidad_control and normalidad_parkinson:
            homocedasticidad = verificar_homocedasticidad(datos_control, datos_parkinson)
            if homocedasticidad:
                p_valor = realizar_prueba_t(datos_control, datos_parkinson, var_equal=True)
                metodo = 'Prueba t (varianzas iguales)'
            else:
                p_valor = realizar_prueba_t(datos_control, datos_parkinson, var_equal=False)
                metodo = 'Prueba t (varianzas desiguales)'
        else:
            p_valor = realizar_prueba_mann_whitney(datos_control, datos_parkinson)
            metodo = 'Prueba U de Mann-Whitney'

        # Almacenar los resultados en una lista
        resultados.append({
            'Canal': canal + 1,
            'Método': metodo,
            'p-valor': p_valor
        })

    resultados_df = pd.DataFrame(resultados)

    # Imprimir resultados
    print("Resultados del análisis estadístico:")
    print(resultados_df)

    # Guardar resultados en un archivo CSV
    resultados_df.to_csv('resultados_analisis.csv', index=False)

    return resultados_df

# Supongamos que df_control y df_parkinson están definidos
resultados_analisis = analizar_canales(df_control, df_parkinson)


Resultados del análisis estadístico:
      Canal                    Método   p-valor
0         1  Prueba U de Mann-Whitney  0.414533
1         2  Prueba U de Mann-Whitney  0.405701
2         3  Prueba U de Mann-Whitney  0.388376
3         4  Prueba U de Mann-Whitney  0.388376
4         5  Prueba U de Mann-Whitney  0.396981
...     ...                       ...       ...
1995   1996  Prueba U de Mann-Whitney  0.405701
1996   1997  Prueba U de Mann-Whitney  0.414533
1997   1998  Prueba U de Mann-Whitney  0.396981
1998   1999  Prueba U de Mann-Whitney  0.405701
1999   2000  Prueba U de Mann-Whitney  0.432537

[2000 rows x 3 columns]
